# Vanilla Binary RNN Model

This notebook is used to train a simple Vanilla RNN model for Binary classification of Piano and Electric Guitar. We chose the following two classes because there numbers are really close to each other, hence would be able to solve the class imbalance issue beforehand and dont have to worry about it.

### Importing Required Libraries

This section will import the required libaries that will be used to actually implement the training for the Vanilla RNN Binary Classification Model

In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
from py_scripts.raw_training_data_creation import *

In [1]:
import numpy as np
import pandas as pd
import os
import glob
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Input
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)

C:\Users\Saksham Goel\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Training Data Creator

This section uses the script from the training data creation module to load the training data

In [18]:
import numpy as np
import os
import glob
from scipy.io import wavfile
from typing import List


def exist_directory(directory_path: str):
    return os.path.isdir(directory_path)


def exist_file(file_path: str):
    return os.path.isfile(file_path)


def get_directory_contents(directory_path: str, pattern: str):
    if exist_directory(directory_path):
        # Get the current working directory
        cwd = os.getcwd()
        # Change the directory into the target directory path
        os.chdir(directory_path)
        # Get the list of directory contents
        directory_contents = glob.glob(pattern)
        # Change back to the original process working directory
        os.chdir(cwd)
        # Return back the directory contents
        return directory_contents
    else:
        return []
    

def get_subdirectory_names(directory_path: str):
    return list(map(lambda subdirectory_name: subdirectory_name.replace('\\', '').replace('//', ''),
                    get_directory_contents(directory_path, '*//')))


def get_file_names(directory_path: str, file_extension: str):
    return get_directory_contents(directory_path, f'*{file_extension}')


def construct_path(directory_path: str, directory_content_name: str):
    return os.path.join(directory_path, directory_content_name)


def read_wav_file(file_path: str):
    if exist_file(file_path):
        return wavfile.read(file_path)
    else:
        return 44100, np.array([])
    

def get_sound_signals(wav_file_data):
    return wav_file_data[-1]


def shift_sound_signals(sound_signals: np.array):
    return np.int32(sound_signals + 2**15)


def normalize_sound_signals(sound_signals: np.array):
    return np.float32(sound_signals / (2. ** 15))


def get_left_channel_data(sound_signals: np.array):
    return sound_signals[:, 0]


def get_right_channel_data(sound_signals: np.array):
    return sound_signals[:, -1]


def get_sound_feature_vectors_from_file(file_path: str, normalize: bool = True, shift: bool = False):
    if normalize:
        sound_signals = normalize_sound_signals(get_sound_signals(read_wav_file(file_path)))
    else:
        sound_signals = get_sound_signals(read_wav_file(file_path))
        if shift:
            sound_signals = shift_sound_signals(sound_signals)
    left_channel_feature_vector, right_channel_feature_vector = \
        get_left_channel_data(sound_signals), get_right_channel_data(sound_signals)
    return left_channel_feature_vector, right_channel_feature_vector


def stack_data(feature_matrix: np.array, left_channel_features: np.array, right_channel_features: np.array):
    if len(feature_matrix) == 0:
        return np.stack((left_channel_features, right_channel_features))
    else:
        return np.vstack((feature_matrix, left_channel_features, right_channel_features))


def get_class_data(parent_directory_path: str, class_label: str, number_of_examples: int = 0,
                   normalize: bool = True, shift: bool = False):
    # Initializing the class feature matrix and target vector
    class_feature_matrix = []
    class_target_vector = []
    # Construct the path
    class_directory_path = construct_path(parent_directory_path, class_label)
    # Make sure the given path is correct
    if not exist_directory(class_directory_path):
        return np.array(class_feature_matrix), np.array(class_target_vector)
    
    # Get the names of the wav file belonging to the current class
    wav_file_names = set(get_file_names(class_directory_path, '.wav'))
    # Get the subset of the classes, if want only limited number of training examples
    if number_of_examples:
        wav_file_names = wav_file_names[:np.abs(number_of_examples)]
    print(f'Processing: {len(wav_file_names)} files')
    # Iterate through each wav file
    for file_name in wav_file_names:
        file_path = construct_path(class_directory_path, file_name)
        left_channel_features, right_channel_features = \
            get_sound_feature_vectors_from_file(file_path, normalize, shift)
        #class_feature_matrix += [left_channel_features[:44100], right_channel_features[:44100]]
        class_feature_matrix += [left_channel_features[:88200], right_channel_features[:88200]]
        class_target_vector += [class_label]*2
        
    return np.array(class_feature_matrix), np.array(class_target_vector)


def load_irmas_data(parent_directory_path: str, class_labels_to_process: List[str], 
                    number_of_training_examples_per_class: int = 0, normalize: bool = True,
                    shift: bool = False):
    if not exist_directory(parent_directory_path):
        print(f'Invalid directory: {parent_directory_path}')
    
    class_labels = class_labels_to_process if class_labels_to_process else get_subdirectory_names(parent_directory_path)
    
    feature_matrix = np.array([])
    target_vector = np.array([])
    
    for class_label in class_labels:
        print(f'Getting Data from {class_label}')
        class_feature_matrix, class_target_vector = get_class_data(parent_directory_path, class_label, 
                                                                   number_of_training_examples_per_class, normalize,
                                                                   shift)
        print(f'Loaded all the data from the class')
        if feature_matrix.size:
            feature_matrix = np.vstack((feature_matrix, class_feature_matrix))
            target_vector = np.hstack((target_vector, class_target_vector))
        else:
            feature_matrix = class_feature_matrix
            target_vector = class_target_vector
    
    return feature_matrix, target_vector

In [19]:
X_train, y_train = load_irmas_data(f'../../data/whole_dataset/training/', 
                                   ['pia', 'gac'], normalize = True, shift = False)

Getting Data from pia
Processing: 721 files
Loaded all the data from the class
Getting Data from gac
Processing: 637 files
Loaded all the data from the class


In [20]:
print(f'Total Number of Training Samples: {y_train.shape}')
print(f'Total number of features for each sample: {X_train.shape[-1]}')
print(f'Minimum Feature Value: {np.min(X_train)}, Maximum Feature Value: {np.max(X_train)}')

Total Number of Training Samples: (2716,)
Total number of features for each sample: 88200
Minimum Feature Value: -1.0, Maximum Feature Value: 0.999969482421875


In [21]:
y_train_categorical = pd.Categorical(y_train)
y_train_numerical = y_train_categorical.codes

### Model Definition

This section will define the model architecture that will be used for the training purposes

In [6]:
# Defining the parameters for the Embedding layer
number_of_features = 441
embedding_vector_length = 10
highest_val = np.max(X_train)+1

In [22]:
X_train_reshaped = X_train.reshape((2716, 126, 700))

In [8]:
model = Sequential()
model.add(SimpleRNN(50, input_shape=(147,300), dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 50)                17550     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 17,601
Trainable params: 17,601
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(X_train_reshaped, y_train_numerical, epochs=20, batch_size=64)

Epoch 1/20
2716/2716 [==============================] - 4s 2ms/step - loss: 0.7024 - acc: 0.5221
Epoch 2/20
2716/2716 [==============================] - 4s 1ms/step - loss: 0.6716 - acc: 0.5924
Epoch 3/20
2716/2716 [==============================] - 4s 1ms/step - loss: 0.6642 - acc: 0.5946
Epoch 4/20
2716/2716 [==============================] - 4s 1ms/step - loss: 0.6506 - acc: 0.6197
Epoch 5/20
2716/2716 [==============================] - 4s 1ms/step - loss: 0.6458 - acc: 0.6230
Epoch 6/20
2716/2716 [==============================] - 4s 1ms/step - loss: 0.6335 - acc: 0.6432
Epoch 7/20
2716/2716 [==============================] - 4s 1ms/step - loss: 0.6492 - acc: 0.6156
Epoch 8/20
2716/2716 [==============================] - 4s 1ms/step - loss: 0.6277 - acc: 0.6451
Epoch 9/20
2716/2716 [==============================] - 4s 1ms/step - loss: 0.6133 - acc: 0.6701
Epoch 10/20
2716/2716 [==============================] - 4s 2ms/step - loss: 0.6152 - acc: 0.6587
Epoch 11/20
2716/2716 [======

In [17]:
# Final evaluation of the model
scores = model.evaluate(X_train_reshaped, y_train_numerical, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

2716/2716 [==============================] - 3s 1ms/step
Accuracy: 77.47%


# Model Trained with More Data

In [23]:
model = Sequential()
model.add(SimpleRNN(50, input_shape=(126,700), dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 50)                37550     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 37,601
Trainable params: 37,601
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(X_train_reshaped, y_train_numerical, epochs=20, batch_size=64)

Epoch 1/20
2716/2716 [==============================] - 7s 3ms/step - loss: 0.6921 - acc: 0.5486
Epoch 2/20
2716/2716 [==============================] - 6s 2ms/step - loss: 0.6416 - acc: 0.6395
Epoch 3/20
2716/2716 [==============================] - 6s 2ms/step - loss: 0.6193 - acc: 0.6513
Epoch 4/20
2716/2716 [==============================] - 6s 2ms/step - loss: 0.5896 - acc: 0.6874
Epoch 5/20
2716/2716 [==============================] - 6s 2ms/step - loss: 0.5615 - acc: 0.7187
Epoch 6/20
2716/2716 [==============================] - 6s 2ms/step - loss: 0.5573 - acc: 0.7139
Epoch 7/20
2716/2716 [==============================] - 6s 2ms/step - loss: 0.5211 - acc: 0.7496
Epoch 8/20
2716/2716 [==============================] - 6s 2ms/step - loss: 0.4711 - acc: 0.7813
Epoch 9/20
2716/2716 [==============================] - 6s 2ms/step - loss: 0.4619 - acc: 0.7732
Epoch 10/20
2716/2716 [==============================] - 7s 2ms/step - loss: 0.4495 - acc: 0.7898
Epoch 11/20
2716/2716 [======

In [25]:
# Final evaluation of the model
scores = model.evaluate(X_train_reshaped, y_train_numerical, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

2716/2716 [==============================] - 6s 2ms/step
Accuracy: 86.16%
